# Aim of Project and Setup

**Note:** *This version does not include any fielding statistics for players*

This notebook will attempt to create a way to make an **MLB Top 100 Players List** by using various *player statistics* and *MVP voting results* from prior years.

A linear regression model will be employed to predict the amount of Vote Pts a player will get in MVP voting from a certain player's statistics. To train this model, Initially, I will be using data from the last 10 MVP voting scenarios (2010 - 2019).

Once I get a relatively accurate model for predicting Vote Pts, The equation learned by the model will be applied to the 2019 Regular Season Stats of all players, not just ones who were included in MVP voting. The model will predict Vote Pts, which players will be sorted by to create a Top 100 Players List.

This List will then be compared to the Top 100 Players List that MLB Network put out in February.



In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
path_to_mvp_data = '../data/mvp_voting_2010s/'

# Gathering and Joining the Data



Data is from baseball-reference.com at this link https://www.baseball-reference.com/awards/mvp.shtml.  I will be employing the last 10 years of MVP Voting for both the American Leaugue and National League. Thank you Sports Reference!

In [8]:
def rename_columns(df):
  df.columns = ['Rank', 'Name', 'Team', 'Vote Pts', '1st Place', 'Share', 'WAR', 'G', 'AB', 'R', 'H', 'HR', 'RBI', 'SB', 'BB', 'BA', 'OBP', 'SLG', 'OPS', 'W', 'L', 'ERA', 'WHIP', 'G(P)', 'GS', 'SV', 'IP', 'H(P)', 'HR(P)', 'BB(P)', 'SO']
  df.drop(0, inplace=True)
  return df

In [9]:
df_al_mvp_2019 = pd.read_csv(path_to_mvp_data + 'AL MVP Voting 2019.csv')
rename_columns(df_al_mvp_2019)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Mike Trout,LAA,355,17,85%,8.2,134,470,110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Alex Bregman,HOU,335,13,80%,9.1,156,554,122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Marcus Semien,OAK,228,0,54%,8.9,162,657,123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,DJ LeMahieu,NYY,178,0,42%,5.9,145,602,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Xander Bogaerts,BOS,147,0,35%,5.9,155,614,110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Matt Chapman,OAK,89,0,21%,8.3,156,583,102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,George Springer,HOU,69,0,16%,6.4,122,479,96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Mookie Betts,BOS,67,0,16%,6.9,150,597,135,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Nelson Cruz,MIN,62,0,15%,4.4,120,454,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Gerrit Cole,HOU,61,0,15%,6.5,3,7,0,...,2.5,0.895,33,33,0,212.1,142,29,48,326


In [10]:
df_nl_mvp_2019 = pd.read_csv(path_to_mvp_data + 'NL MVP Voting 2019.csv')
rename_columns(df_nl_mvp_2019)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Cody Bellinger,LAD,362,19,86%,9.1,156,558,121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Christian Yelich,MIL,317,10,75%,7,130,489,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Anthony Rendon,WSN,242,1,58%,6.4,146,545,117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Ketel Marte,ARI,198,0,47%,7.2,144,569,97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Ronald Acuna Jr.,ATL,155,0,37%,5.7,156,626,127,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Nolan Arenado,COL,120,0,29%,6.7,155,588,102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Pete Alonso,NYM,102,0,24%,5.2,161,597,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Freddie Freeman,ATL,90,0,21%,4,158,597,113,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Juan Soto,WSN,45,0,11%,4.6,150,542,110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Jacob deGrom,NYM,44,0,10%,8.2,31,65,4,...,2.43,0.971,32,32,0,204,154,19,44,255


In [11]:
df_al_mvp_2018 = pd.read_csv(path_to_mvp_data + 'AL MVP Voting 2018.csv')
rename_columns(df_al_mvp_2018)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Mookie Betts,BOS,410,28,98%,10.6,136,520,129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Mike Trout,LAA,265,1,63%,10.2,140,471,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Jose Ramirez,CLE,208,0,50%,7.6,157,578,110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,J.D. Martinez,BOS,198,1,47%,6.5,150,569,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Alex Bregman,HOU,192,0,46%,7.3,157,594,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Francisco Lindor,CLE,169,0,40%,7.8,158,661,129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Matt Chapman,OAK,141,0,34%,8.3,145,547,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Khris Davis,OAK,41,0,10%,2.9,151,576,98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Blake Snell,TBR,38,0,9%,7.1,1,1,0,...,1.89,0.974,31,31,0,180.2,112,16,64,221
10,10,Justin Verlander,HOU,26,0,6%,6.8,2,5,0,...,2.52,0.902,34,34,0,214,156,28,37,290


In [12]:
df_nl_mvp_2018 = pd.read_csv(path_to_mvp_data + 'NL MVP Voting 2018.csv')
rename_columns(df_nl_mvp_2018)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Christian Yelich,MIL,415,29,99%,7.3,147,574,118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Javier Baez,CHC,250,0,60%,5.8,160,606,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Nolan Arenado,COL,203,0,48%,5.9,156,590,104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Freddie Freeman,ATL,174,0,41%,5.6,162,618,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Jacob deGrom,NYM,141,1,34%,10.3,30,67,1,...,1.7,0.912,32,32,0,217,152,10,46,269
6,6,Paul Goldschmidt,ARI,115,0,27%,6.2,158,593,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Lorenzo Cain,MIL,109,0,26%,6.7,141,539,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Trevor Story,COL,108,0,26%,5.6,157,598,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Matt Carpenter,STL,105,0,25%,4.9,156,564,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Max Scherzer,WSN,59,0,14%,9.2,32,70,8,...,2.53,0.911,33,33,0,220.2,150,23,51,300


In [13]:
df_al_mvp_2017 = pd.read_csv(path_to_mvp_data + 'AL MVP Voting 2017.csv')
rename_columns(df_al_mvp_2017)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Jose Altuve,HOU,405,27,96%,7.6,153,590,112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Aaron Judge,NYY,279,2,66%,7.9,155,542,128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Jose Ramirez,CLE,237,1,56%,7,152,585,107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Mike Trout,LAA,197,0,47%,6.7,114,402,92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Francisco Lindor,CLE,143,0,34%,5.7,159,651,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Mookie Betts,BOS,110,0,26%,6.3,153,628,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Corey Kluber,CLE,101,0,24%,7.9,1,2,0,...,2.25,0.869,29,29,0,203.2,141,21,36,265
8,8,Andrelton Simmons,LAA,60,0,14%,7.8,158,589,77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Chris Sale,BOS,56,0,13%,6.1,1,3,0,...,2.9,0.97,32,32,0,214.1,165,24,43,308
10,10,Nelson Cruz,SEA,44,0,10%,4.2,155,556,91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_nl_mvp_2017 = pd.read_csv(path_to_mvp_data + 'NL MVP Voting 2017.csv')
rename_columns(df_nl_mvp_2017)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Giancarlo Stanton,MIA,302,10,72%,8,159,597,123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Joey Votto,CIN,300,10,71%,8.1,162,559,106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Paul Goldschmidt,ARI,239,4,57%,6.4,155,558,117,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Nolan Arenado,COL,229,2,55%,6.6,159,606,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Charlie Blackmon,COL,205,3,49%,5.5,159,644,137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Anthony Rendon,WSN,141,0,34%,6,147,508,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Kris Bryant,CHC,132,1,31%,5.6,151,549,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Justin Turner,LAD,43,0,10%,5.6,130,457,72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Cody Bellinger,LAD,38,0,9%,4,132,480,87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Max Scherzer,WSN,34,0,8%,7.5,30,62,4,...,2.51,0.902,31,31,0,200.2,126,22,55,268


In [15]:
df_al_mvp_2016 = pd.read_csv(path_to_mvp_data + 'AL MVP Voting 2016.csv')
rename_columns(df_al_mvp_2016)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Mike Trout,LAA,356,19,85%,10.5,159,549,123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Mookie Betts,BOS,311,9,74%,9.5,158,672,122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Jose Altuve,HOU,227,0,54%,7.9,161,640,108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Josh Donaldson,TOR,200,0,48%,7.2,155,577,122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Manny Machado,BAL,150,0,36%,7.3,157,640,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,David Ortiz,BOS,147,1,35%,5.2,151,537,79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Adrian Beltre,TEX,135,1,32%,6.7,153,583,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Robinson Cano,SEA,79,0,19%,7.3,161,655,107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Miguel Cabrera,DET,56,0,13%,5.1,158,595,92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,9,Francisco Lindor,CLE,56,0,13%,5.1,158,604,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_nl_mvp_2016 = pd.read_csv(path_to_mvp_data + 'NL MVP Voting 2016.csv')
rename_columns(df_nl_mvp_2016)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Kris Bryant,CHC,415,29,99%,7.3,155,603,121,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Daniel Murphy,WSN,245,1,58%,4.5,142,531,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Corey Seager,LAD,240,0,57%,5.2,157,627,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Anthony Rizzo,CHC,202,0,48%,5.8,155,583,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Nolan Arenado,COL,199,0,47%,5.9,160,618,116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Freddie Freeman,ATL,129,0,31%,6.3,158,589,102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Joey Votto,CIN,100,0,24%,4.2,158,556,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Yoenis CÃ©spedes,NYM,45,0,11%,3,132,479,72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Justin Turner,LAD,44,0,10%,5,151,556,79,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Max Scherzer,WSN,39,0,9%,6.4,33,70,4,...,2.96,0.968,34,34,0,228.1,165,31,56,284


In [17]:
df_al_mvp_2015 = pd.read_csv(path_to_mvp_data + 'AL MVP Voting 2015.csv')
rename_columns(df_al_mvp_2015)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Josh Donaldson,TOR,385,23,92%,7.1,158,620,122,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Mike Trout,LAA,304,7,72%,9.6,159,575,104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Lorenzo Cain,KCR,225,0,54%,7,140,551,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Manny Machado,BAL,158,0,38%,7.5,162,633,102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Dallas Keuchel,HOU,107,0,25%,6.6,2,5,0,...,2.48,1.017,33,33,0,232,185,17,51,216
6,6,Nelson Cruz,SEA,94,0,22%,5.1,152,590,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Adrian Beltre,TEX,83,0,20%,4.4,143,567,83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Jose Bautista,TOR,82,0,20%,4.8,153,543,108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,David Price,TOT,62,0,15%,6.1,3,9,0,...,2.45,1.076,32,32,0,220.1,190,17,47,225
10,10,Jose Altuve,HOU,44,0,10%,4,154,638,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_nl_mvp_2015 = pd.read_csv(path_to_mvp_data + 'NL MVP Voting 2015.csv')
rename_columns(df_nl_mvp_2015)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Bryce Harper,WSN,420,30,100%,9.7,153,521,118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Paul Goldschmidt,ARI,234,0,56%,8.3,159,567,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Joey Votto,CIN,175,0,42%,7.8,158,545,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Anthony Rizzo,CHC,162,0,39%,6.4,160,586,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Andrew McCutchen,PIT,139,0,33%,5.1,157,566,91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Jake Arrieta,CHC,134,0,32%,8.6,30,79,5,...,1.77,0.865,33,33,0,229,150,10,48,236
7,7,Zack Greinke,LAD,130,0,31%,9.5,30,67,8,...,1.66,0.844,32,32,0,222.2,148,14,40,200
8,8,Nolan Arenado,COL,102,0,24%,6.3,157,616,97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Buster Posey,SFG,84,0,20%,6,150,557,74,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Clayton Kershaw,LAD,49,0,12%,7.3,31,71,2,...,2.13,0.881,33,33,0,232.2,163,15,42,301


In [21]:
df_al_mvp_2014 = pd.read_csv(path_to_mvp_data + 'AL MVP Voting 2014.csv')
rename_columns(df_al_mvp_2014)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Mike Trout,LAA,420,30,100%,7.7,157,602,115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Victor Martinez,DET,229,0,55%,5.5,151,561,87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Michael Brantley,CLE,191,0,45%,7,156,611,94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,JosÃ© Abreu,CHW,145,0,35%,5.8,145,556,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Robinson Cano,SEA,124,0,30%,6.3,157,595,77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Jose Bautista,TOR,122,0,29%,6.9,155,553,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Nelson Cruz,BAL,102,0,24%,4.5,159,613,87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Josh Donaldson,OAK,96,0,23%,6.9,158,608,93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Miguel Cabrera,DET,82,0,20%,5.1,159,611,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Felix Hernandez,SEA,48,0,11%,6.3,1,2,0,...,2.14,0.915,34,34,0,236,170,16,46,248


In [22]:
df_nl_mvp_2014 = pd.read_csv(path_to_mvp_data + 'NL MVP Voting 2014.csv')
rename_columns(df_nl_mvp_2014)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Clayton Kershaw,LAD,355,18,85%,8.2,26,63,5,...,1.77,0.857,27,27,0,198.1,139,9,31,239
2,2,Giancarlo Stanton,MIA,298,8,71%,6.5,145,539,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Andrew McCutchen,PIT,271,4,65%,6.4,146,548,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Jonathan Lucroy,MIL,167,0,40%,6.4,153,585,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Anthony Rendon,WSN,155,0,37%,6.5,153,613,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Buster Posey,SFG,152,0,36%,5.1,147,547,72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Adrian Gonzalez,LAD,57,0,14%,4.4,159,591,83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Adam Wainwright,STL,53,0,13%,6.4,30,72,4,...,2.38,1.031,32,32,0,227,184,10,50,179
9,9,Josh Harrison,PIT,52,0,12%,5.6,143,520,77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Anthony Rizzo,CHC,37,0,9%,5.6,140,524,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_al_mvp_2013 = pd.read_csv(path_to_mvp_data + 'AL MVP Voting 2013.csv')
rename_columns(df_al_mvp_2013)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Miguel Cabrera,DET,385,23,92%,7.5,148,555,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Mike Trout,LAA,282,5,67%,8.9,157,589,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Chris Davis,BAL,232,1,55%,7.1,160,584,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Josh Donaldson,OAK,222,1,53%,7.2,158,579,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Robinson Cano,NYY,150,0,36%,6.6,160,605,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Evan Longoria,TBR,103,0,25%,5.8,160,614,91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Adrian Beltre,TEX,99,0,24%,5.7,161,631,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,7,Dustin Pedroia,BOS,99,0,24%,6.1,160,641,91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Manny Machado,BAL,57,0,14%,5.9,156,667,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,David Ortiz,BOS,47,0,11%,4.4,137,518,84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
df_nl_mvp_2013 = pd.read_csv(path_to_mvp_data + 'NL MVP Voting 2013.csv')
rename_columns(df_nl_mvp_2013)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Andrew McCutchen,PIT,409,28,97%,7.8,157,583,97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Paul Goldschmidt,ARI,242,0,58%,6.3,160,602,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Yadier Molina,STL,219,2,52%,6.2,136,505,68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Matt Carpenter,STL,194,0,46%,6.6,157,626,126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Freddie Freeman,ATL,154,0,37%,5.3,147,551,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Joey Votto,CIN,149,0,35%,6.6,162,581,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Clayton Kershaw,LAD,146,0,35%,8.6,35,77,8,...,1.83,0.915,33,33,0,236,164,11,52,232
8,8,Hanley Ramirez,LAD,58,0,14%,5.2,86,304,62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Carlos Gomez,MIL,43,0,10%,7.6,147,536,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Jay Bruce,CIN,30,0,7%,5.2,160,626,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_al_mvp_2012 = pd.read_csv(path_to_mvp_data + 'AL MVP Voting 2012.csv')
rename_columns(df_al_mvp_2012)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Miguel Cabrera,DET,362,22,92%,7.1,161,622,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Mike Trout,LAA,281,6,72%,10.5,139,559,129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Adrian Beltre,TEX,210,0,54%,7.2,156,604,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Robinson Cano,NYY,149,0,38%,8.4,161,627,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Josh Hamilton,TEX,127,0,32%,4,148,562,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Adam Jones,BAL,120,0,31%,4.1,162,648,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Derek Jeter,NYY,77,0,20%,2.2,159,683,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Justin Verlander,DET,58,0,15%,8,3,4,0,...,2.64,1.057,33,33,0,238.1,192,19,60,239
9,9,Prince Fielder,DET,56,0,14%,4.7,162,581,83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Yoenis CÃ©spedes,OAK,41,0,10%,4,129,487,70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df_nl_mvp_2012 = pd.read_csv(path_to_mvp_data + 'NL MVP Voting 2012.csv')
rename_columns(df_nl_mvp_2012)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Buster Posey,SFG,422,27,94%,7.6,148,530,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Ryan Braun,MIL,285,3,64%,6.9,154,598,108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Andrew McCutchen,PIT,245,0,55%,6.9,157,593,107,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Yadier Molina,STL,241,2,54%,7.2,138,505,65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Chase Headley,SDP,127,0,28%,6.4,161,604,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Adam LaRoche,WSN,86,0,19%,4.2,154,571,76,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6,David Wright,NYM,86,0,19%,7.1,156,581,91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Craig Kimbrel,ATL,73,0,16%,3.2,59,0,0,...,1.01,0.654,63,0,42,62.2,27,3,14,116
9,9,Aramis Ramirez,MIL,47,0,10%,5.6,149,570,92,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Jay Bruce,CIN,46,0,10%,2.2,155,560,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_al_mvp_2011 = pd.read_csv(path_to_mvp_data + 'AL MVP Voting 2011.csv')
rename_columns(df_al_mvp_2011)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Justin Verlander,DET,280,13,71%,8.6,1,4,0,...,2.4,0.92,34,34,0,251,174,24,57,250
2,2,Jacoby Ellsbury,BOS,242,4,62%,8.3,158,660,119,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Jose Bautista,TOR,231,5,59%,8.3,149,513,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Curtis Granderson,NYY,215,3,55%,6.1,156,583,136,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Miguel Cabrera,DET,193,2,49%,7.6,161,572,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Robinson Cano,NYY,112,0,29%,5.8,159,623,104,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Adrian Gonzalez,BOS,105,0,27%,6.9,159,630,108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Michael Young,TEX,96,1,24%,2.7,159,631,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Dustin Pedroia,BOS,48,0,12%,8,159,635,102,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Evan Longoria,TBR,27,0,7%,7.2,133,483,78,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_nl_mvp_2011 = pd.read_csv(path_to_mvp_data + 'NL MVP Voting 2011.csv')
rename_columns(df_nl_mvp_2011)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Ryan Braun,MIL,388,20,87%,7.7,150,563,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Matt Kemp,LAD,332,10,74%,8,161,602,115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Prince Fielder,MIL,229,1,51%,4.5,162,569,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Justin Upton,ARI,214,1,48%,5.5,159,592,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Albert Pujols,STL,166,0,37%,5.3,147,579,105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Joey Votto,CIN,135,0,30%,6.6,161,599,101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Lance Berkman,STL,118,0,26%,3.8,145,488,90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Troy Tulowitzki,COL,69,0,15%,6.2,143,537,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Roy Halladay,PHI,52,0,12%,8.6,31,74,3,...,2.35,1.04,32,32,0,233.2,208,10,35,220
10,10,Ryan Howard,PHI,39,0,9%,1.2,152,557,81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_al_mvp_2010 = pd.read_csv(path_to_mvp_data + 'AL MVP Voting 2010.csv')
rename_columns(df_al_mvp_2010)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Josh Hamilton,TEX,358,22,91%,8.7,133,518,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Miguel Cabrera,DET,262,5,67%,6.5,150,548,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Robinson Cano,NYY,229,0,58%,8.1,160,626,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Jose Bautista,TOR,165,1,42%,7,161,569,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Paul Konerko,CHW,130,0,33%,4.7,149,548,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Evan Longoria,TBR,100,0,26%,8.2,151,574,96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Carl Crawford,TBR,98,0,25%,7,154,600,110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Joe Mauer,MIN,97,0,25%,5.9,137,510,88,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Adrian Beltre,BOS,83,0,21%,7.8,154,589,84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Delmon Young,MIN,44,0,11%,1.9,153,570,77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df_nl_mvp_2010 = pd.read_csv(path_to_mvp_data + 'NL MVP Voting 2010.csv')
rename_columns(df_nl_mvp_2010)

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
1,1,Joey Votto,CIN,443,31,99%,7,150,547,106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Albert Pujols,STL,279,1,62%,7.5,159,587,115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Carlos Gonzalez,COL,240,0,54%,5.9,145,587,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Adrian Gonzalez,SDP,197,0,44%,4.4,160,591,87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Troy Tulowitzki,COL,132,0,29%,6.7,122,470,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,Roy Halladay,PHI,130,0,29%,8.3,31,92,6,...,2.44,1.041,33,33,0,250.2,231,24,30,219
7,7,Aubrey Huff,SFG,70,0,16%,5.7,157,569,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Jayson Werth,PHI,52,0,12%,4.5,156,554,106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Martin Prado,ATL,51,0,11%,5,140,599,100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,10,Ryan Howard,PHI,50,0,11%,1.2,143,550,87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df_al_mvp_last10 = pd.concat([df_al_mvp_2010, df_al_mvp_2011, df_al_mvp_2012, df_al_mvp_2013, df_al_mvp_2014, df_al_mvp_2015, df_al_mvp_2016, df_al_mvp_2017, df_al_mvp_2018, df_al_mvp_2019], ignore_index=True)
df_al_mvp_last10

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
0,1,Josh Hamilton,TEX,358,22,91%,8.7,133,518,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Miguel Cabrera,DET,262,5,67%,6.5,150,548,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Robinson Cano,NYY,229,0,58%,8.1,160,626,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Jose Bautista,TOR,165,1,42%,7,161,569,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Paul Konerko,CHW,130,0,33%,4.7,149,548,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,21,J.D. Martinez,BOS,1,0,0%,3.4,146,575,98,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,21,Yoan Moncada,CHW,1,0,0%,4.8,132,511,83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
236,21,Charlie Morton,TBR,1,0,0%,4.9,1,2,0,...,3.05,1.084,33,33,0,194.2,154,15,57,240
237,21,Matt Olson,OAK,1,0,0%,5.4,127,483,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_nl_mvp_last10 = pd.concat([df_nl_mvp_2010, df_nl_mvp_2011, df_nl_mvp_2012, df_nl_mvp_2013, df_nl_mvp_2014, df_nl_mvp_2015, df_nl_mvp_2016, df_nl_mvp_2017, df_nl_mvp_2018, df_nl_mvp_2019], ignore_index=True)
df_nl_mvp_last10

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
0,1,Joey Votto,CIN,443,31,99%,7,150,547,106,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Albert Pujols,STL,279,1,62%,7.5,159,587,115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Carlos Gonzalez,COL,240,0,54%,5.9,145,587,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Adrian Gonzalez,SDP,197,0,44%,4.4,160,591,87,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Troy Tulowitzki,COL,132,0,29%,6.7,122,470,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,19,Hyun Jin Ryu,LAD,3,0,1%,5,28,51,3,...,2.32,1.007,29,29,0,182.2,160,17,24,163
245,20,Paul Goldschmidt,STL,2,0,0%,2.4,161,597,97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,20,Kolten Wong,STL,2,0,0%,5.2,148,478,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,22,Kevin Pillar,SFG,1,0,0%,1.3,156,595,82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df_mvp_last10 = pd.concat([df_al_mvp_last10, df_nl_mvp_last10], ignore_index=True)
df_mvp_last10

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
0,1,Josh Hamilton,TEX,358,22,91%,8.7,133,518,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Miguel Cabrera,DET,262,5,67%,6.5,150,548,111,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Robinson Cano,NYY,229,0,58%,8.1,160,626,103,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Jose Bautista,TOR,165,1,42%,7,161,569,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Paul Konerko,CHW,130,0,33%,4.7,149,548,89,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,19,Hyun Jin Ryu,LAD,3,0,1%,5,28,51,3,...,2.32,1.007,29,29,0,182.2,160,17,24,163
484,20,Paul Goldschmidt,STL,2,0,0%,2.4,161,597,97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
485,20,Kolten Wong,STL,2,0,0%,5.2,148,478,61,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,22,Kevin Pillar,SFG,1,0,0%,1.3,156,595,82,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Cleaning Data to Separate Batters and Pitchers

As far as I know, there is no way to run a single model on two different training and test sets. I will need to find a way to separate batters and pitchers, run two separate models and rejoin their results.

In [35]:
# Converting numeric dtypes to float
df_mvp_last10 = df_mvp_last10.fillna(0)
convert_dict = {'Vote Pts': int,
                '1st Place': int,
                'WAR': float,
                'G': int,
                'AB': int,
                'R': int,
                'H': int,
                'HR': int,
                'RBI': int,
                'SB': int,
                'BB': int,
                'BA': float,
                'OBP': float,
                'SLG': float,
                'OPS': float,
                'W': int,
                'L': int,
                'ERA': float,
                'WHIP': float,
                'G(P)': int,
                'GS': int,
                'SV': int,
                'IP': float,
                'H(P)': int,
                'HR(P)': int,
                'BB(P)': int,
                'SO': int
                }
df_mvp_last10 = df_mvp_last10.astype(convert_dict)
df_mvp_last10

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
0,1,Josh Hamilton,TEX,358,22,91%,8.7,133,518,95,...,0.00,0.000,0,0,0,0.0,0,0,0,0
1,2,Miguel Cabrera,DET,262,5,67%,6.5,150,548,111,...,0.00,0.000,0,0,0,0.0,0,0,0,0
2,3,Robinson Cano,NYY,229,0,58%,8.1,160,626,103,...,0.00,0.000,0,0,0,0.0,0,0,0,0
3,4,Jose Bautista,TOR,165,1,42%,7.0,161,569,109,...,0.00,0.000,0,0,0,0.0,0,0,0,0
4,5,Paul Konerko,CHW,130,0,33%,4.7,149,548,89,...,0.00,0.000,0,0,0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,19,Hyun Jin Ryu,LAD,3,0,1%,5.0,28,51,3,...,2.32,1.007,29,29,0,182.2,160,17,24,163
484,20,Paul Goldschmidt,STL,2,0,0%,2.4,161,597,97,...,0.00,0.000,0,0,0,0.0,0,0,0,0
485,20,Kolten Wong,STL,2,0,0%,5.2,148,478,61,...,0.00,0.000,0,0,0,0.0,0,0,0,0
486,22,Kevin Pillar,SFG,1,0,0%,1.3,156,595,82,...,0.00,0.000,0,0,0,0.0,0,0,0,0


In [36]:
df_mvp_last10.dtypes

Rank          object
Name          object
Team          object
Vote Pts       int64
1st Place      int64
Share         object
WAR          float64
G              int64
AB             int64
R              int64
H              int64
HR             int64
RBI            int64
SB             int64
BB             int64
BA           float64
OBP          float64
SLG          float64
OPS          float64
W              int64
L              int64
ERA          float64
WHIP         float64
G(P)           int64
GS             int64
SV             int64
IP           float64
H(P)           int64
HR(P)          int64
BB(P)          int64
SO             int64
dtype: object

In [37]:
df_mvp_pitcher_last10 = df_mvp_last10[df_mvp_last10['G(P)'] > 0]
df_mvp_pitcher_last10

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
11,12,Rafael Soriano,TBR,21,0,5%,2.1,7,0,0,...,1.73,0.802,64,0,45,62.1,36,4,14,57
12,13,CC Sabathia,NYY,13,0,3%,4.8,2,5,0,...,3.18,1.191,34,34,0,237.2,209,20,74,197
15,16,Felix Hernandez,SEA,6,0,2%,7.1,1,2,0,...,2.27,1.057,34,34,0,249.2,194,17,70,232
18,19,Joakim Soria,KCR,1,0,0%,3.7,2,0,0,...,1.78,1.051,66,0,43,65.2,53,4,16,71
20,1,Justin Verlander,DET,280,13,71%,8.6,1,4,0,...,2.40,0.920,34,34,0,251.0,174,24,57,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,10,Jacob deGrom,NYM,44,0,10%,8.2,31,65,4,...,2.43,0.971,32,32,0,204.0,154,19,44,255
477,13,Jack Flaherty,STL,9,0,2%,5.8,35,54,4,...,2.75,0.968,33,33,0,196.1,135,25,55,231
481,15,Stephen Strasburg,WSN,4,0,1%,6.4,33,72,4,...,3.32,1.038,33,33,0,209.0,161,24,56,251
483,19,Hyun Jin Ryu,LAD,3,0,1%,5.0,28,51,3,...,2.32,1.007,29,29,0,182.2,160,17,24,163


In [38]:
df_mvp_batter_last10 = df_mvp_last10[df_mvp_last10['G(P)'] == 0]
df_mvp_batter_last10

,Rank,Name,Team,Vote Pts,1st Place,Share,WAR,G,AB,R,...,ERA,WHIP,G(P),GS,SV,IP,H(P),HR(P),BB(P),SO
0,1,Josh Hamilton,TEX,358,22,91%,8.7,133,518,95,...,0.0,0.0,0,0,0,0.0,0,0,0,0
1,2,Miguel Cabrera,DET,262,5,67%,6.5,150,548,111,...,0.0,0.0,0,0,0,0.0,0,0,0,0
2,3,Robinson Cano,NYY,229,0,58%,8.1,160,626,103,...,0.0,0.0,0,0,0,0.0,0,0,0,0
3,4,Jose Bautista,TOR,165,1,42%,7.0,161,569,109,...,0.0,0.0,0,0,0,0.0,0,0,0,0
4,5,Paul Konerko,CHW,130,0,33%,4.7,149,548,89,...,0.0,0.0,0,0,0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,15,Max Muncy,LAD,4,0,1%,5.7,141,487,101,...,0.0,0.0,0,0,0,0.0,0,0,0,0
482,15,Eugenio Suarez,CIN,4,0,1%,3.9,159,575,87,...,0.0,0.0,0,0,0,0.0,0,0,0,0
484,20,Paul Goldschmidt,STL,2,0,0%,2.4,161,597,97,...,0.0,0.0,0,0,0,0.0,0,0,0,0
485,20,Kolten Wong,STL,2,0,0%,5.2,148,478,61,...,0.0,0.0,0,0,0,0.0,0,0,0,0


# Preprocessing and Training the Models

Now that we have our data where we want it, we can start to set up the model. We need to pick our data columns, standardize the data, randomly split the data into train and test, and build the linear regression model.

## 1 - Picking Data Columns

In [39]:
batting_data_columns = ['G', 'AB', 'RBI', 'SB', 'BB', 'BA', 'OBP', 'SLG', 'OPS', 'H', 'HR']
pitching_data_columns = ['W', 'L', 'G(P)', 'GS', 'SV', 'IP', 'HR(P)', 'BB(P)', 'SO', 'ERA', 'WHIP', 'H(P)']
batting_data_x = df_mvp_batter_last10[batting_data_columns]
pitching_data_x = df_mvp_pitcher_last10[pitching_data_columns]

In [40]:
batting_data_y = df_mvp_batter_last10['Vote Pts']
pitching_data_y = df_mvp_pitcher_last10['Vote Pts']

## 2 - Train-Test-Split

In [41]:
x_batting_train, x_batting_test, y_batting_train, y_batting_test = train_test_split(batting_data_x, batting_data_y, test_size=0.2, random_state = 23)
x_pitching_train, x_pitching_test, y_pitching_train, y_pitching_test = train_test_split(pitching_data_x, pitching_data_y, test_size=0.2, random_state = 23)

## 3 - Standardizing the Data

In [42]:
standardizer = StandardScaler()
x_batting_train = standardizer.fit_transform(x_batting_train)
x_batting_test = standardizer.fit_transform(x_batting_test)

x_pitching_train = standardizer.fit_transform(x_pitching_train)
x_pitching_test = standardizer.fit_transform(x_pitching_test)

## 4 - Building the Models

In [43]:
# Linear Regression Batting Model
lin_reg_batting_model = LinearRegression().fit(x_batting_train, y_batting_train)
y_lin_batt_predict = lin_reg_batting_model.predict(x_batting_test)
test_lin_batting_mae = sklearn.metrics.mean_absolute_error(y_batting_test, y_lin_batt_predict)
test_lin_batting_rmse = sklearn.metrics.mean_squared_error(y_batting_test, y_lin_batt_predict)
print('MAE: ' + str(test_lin_batting_mae))
print('RMSE: ' + str(test_lin_batting_rmse))

MAE: 90.72069254348435
RMSE: 14380.540029337075


In [44]:
# Linear Regression Pitching Model
lin_reg_pitching_model = LinearRegression().fit(x_pitching_train, y_pitching_train)
y_pitching_predict = lin_reg_pitching_model.predict(x_pitching_test)
test_lin_pitching_mae = sklearn.metrics.mean_absolute_error(y_pitching_test, y_pitching_predict)
test_lin_pitching_rmse = sklearn.metrics.mean_squared_error(y_pitching_test, y_pitching_predict)
print('MAE: ' + str(test_lin_pitching_mae))
print('RMSE: ' + str(test_lin_pitching_rmse))

MAE: 41.714914916322606
RMSE: 3691.853317011272


In [45]:
# Random Forest Batting Model
forest_batting_model = RandomForestRegressor().fit(x_batting_train, y_batting_train)
y_forest_batt_predict = forest_batting_model.predict(x_batting_test)
forest_batt_mae = sklearn.metrics.mean_absolute_error(y_batting_test, y_forest_batt_predict)
forest_batt_rmse = sklearn.metrics.mean_squared_error(y_batting_test, y_forest_batt_predict)
print('MAE: ' + str(forest_batt_mae))
print('RMSE: ' + str(forest_batt_rmse))

MAE: 50.607468354430374
RMSE: 5358.819196202531


In [46]:
# Random Forest Pitching Model
forest_pitching_model = RandomForestRegressor().fit(x_pitching_train, y_pitching_train)
y_forest_pitch_predict = forest_pitching_model.predict(x_pitching_test)
forest_pitch_mae = sklearn.metrics.mean_absolute_error(y_pitching_test, y_forest_pitch_predict)
forest_pitch_rmse = sklearn.metrics.mean_squared_error(y_pitching_test, y_forest_pitch_predict)
print('MAE: ' + str(forest_pitch_mae))
print('RMSE: ' + str(forest_pitch_rmse))

MAE: 31.652631578947375
RMSE: 4146.8216210526325


In [47]:
#XGBoost Batting model
dtrain_batting = xgb.DMatrix(x_batting_train, y_batting_train, enable_categorical=False)
dtest_batting = xgb.DMatrix(x_batting_test, y_batting_test, enable_categorical=False)

params = {"objective": "reg:squaredlogerror", "tree_method": "hist"}

n=5000
xgb_batting_model = xgb.train(
   params=params,
   dtrain=dtrain_batting,
   num_boost_round=n,
)

y_xgb_batting_predict = xgb_batting_model.predict(dtest_batting)
xgb_batting_mae = sklearn.metrics.mean_absolute_error(y_batting_test, y_xgb_batting_predict)
xgb_batting_rmse = sklearn.metrics.mean_squared_error(y_batting_test, y_xgb_batting_predict)
print('MAE: ' + str(xgb_batting_mae))
print('RMSE: ' + str(xgb_batting_rmse))

MAE: 61.0233679723136
RMSE: 10098.439362342753


In [48]:
#XGBoost Pitching model
dtrain_pitch = xgb.DMatrix(x_pitching_train, y_pitching_train, enable_categorical=False)
dtest_pitch = xgb.DMatrix(x_pitching_test, y_pitching_test, enable_categorical=False)

params = {"objective": "reg:squaredlogerror", "tree_method": "hist"}

n=5000
xgb_pitching_model = xgb.train(
   params=params,
   dtrain=dtrain_pitch,
   num_boost_round=n,
)

y_xgb_pitching_predict = xgb_pitching_model.predict(dtest_pitch)
xgb_pitching_mae = sklearn.metrics.mean_absolute_error(y_pitching_test, y_xgb_pitching_predict)
xgb_pitching_rmse = sklearn.metrics.mean_squared_error(y_pitching_test, y_xgb_pitching_predict)
print('MAE: ' + str(xgb_pitching_mae))
print('RMSE: ' + str(xgb_pitching_rmse))

MAE: 26.23001315719203
RMSE: 4194.181824839483


# Applying the Random Forest Models to the 2019 Regular Season

Now that the models are built we can then predict on new data like player statistics from the 2019 MLB Regular Season for example.

## 1 - Loading in the New Data

In [49]:
batting = pd.read_csv("../data/Batting.csv")

batting['PA'] = batting['AB'] + batting['BB'] + batting['HBP'] + batting['SH'] + batting['SF']
batting['BA'] = round((batting['H'] / batting['AB']), 3)
batting['OBP'] = round((batting['H'] + batting['BB'] + batting['HBP']) / batting['PA'], 3)
batting['SLG'] = round((batting['H'] + batting['2B'] + 2 * batting['3B'] + 3 * batting['HR']) / batting['AB'], 3)
batting['OPS'] = batting['OBP'] + batting['SLG']

batting_2019 = batting[batting["yearID"] == 2019]
# get rid of under 200 ab players
batting_2019 = batting_2019[batting_2019['AB'] > 199]
batting_2019

,playerID,yearID,stint,teamID,lgID,G,G_batting,AB,R,H,...,HBP,SH,SF,GIDP,G_old,PA,BA,OBP,SLG,OPS
272,abreujo02,2019,1,CHA,AL,159,NaN,634,85,180,...,13.0,0.0,10.0,24.0,NaN,693.0,0.284,0.330,0.503,0.833
387,acunaro01,2019,1,ATL,NL,156,NaN,626,127,175,...,9.0,0.0,1.0,8.0,NaN,712.0,0.280,0.365,0.518,0.883
415,adamewi01,2019,1,TBA,AL,152,NaN,531,69,135,...,3.0,3.0,1.0,9.0,NaN,584.0,0.254,0.315,0.418,0.733
558,adamsma01,2019,1,WAS,NL,111,NaN,310,42,70,...,2.0,0.0,1.0,7.0,NaN,333.0,0.226,0.276,0.465,0.741
728,adriaeh01,2019,1,MIN,AL,84,NaN,202,34,55,...,6.0,2.0,4.0,2.0,NaN,234.0,0.272,0.346,0.416,0.762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124547,wolteto01,2019,1,COL,NL,121,NaN,359,42,94,...,8.0,2.0,6.0,9.0,NaN,411.0,0.262,0.336,0.329,0.665
124604,wongko01,2019,1,SLN,NL,148,NaN,478,61,136,...,13.0,6.0,5.0,2.0,NaN,549.0,0.285,0.357,0.423,0.780
125672,yastrmi01,2019,1,SFN,NL,107,NaN,371,64,101,...,4.0,1.0,3.0,4.0,NaN,411.0,0.272,0.333,0.518,0.851
125748,yelicch01,2019,1,MIL,NL,130,NaN,489,100,161,...,8.0,0.0,3.0,8.0,NaN,580.0,0.329,0.429,0.671,1.100


In [51]:
pitching = pd.read_csv("../data/Pitching.csv")

# make the rate statistics
pitching['WHIP'] = round(((pitching['H'] + pitching['BB']) / pitching['IPouts']) * 3, 3)
pitching['G(P)'] = pitching['G']
pitching['H(P)'] = pitching['H']
pitching['HR(P)'] = pitching['HR']
pitching['BB(P)'] = pitching['BB']
pitching['IP'] = (pitching['IPouts'] // 3) + (pitching['IPouts'] % 3) / 10

pitching_2019 = pitching[pitching['yearID'] == 2019]
# get rid of under 20 IP players
pitching_2019 = pitching_2019[pitching_2019['IP'] > 20]
pitching_2019

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,R,SH,SF,GIDP,WHIP,G(P),H(P),HR(P),BB(P),IP
218,adamja01,2019,1,TOR,AL,3,0,23,0,0,...,8,0.0,3.0,2.0,1.154,23,15,1.0,10,21.2
239,adamsau02,2019,2,SEA,AL,2,2,29,2,0,...,13,0.0,1.0,3.0,1.097,29,20,4.0,14,31.0
268,adamsch01,2019,1,NYA,AL,1,1,13,0,0,...,25,0.0,0.0,2.0,1.974,13,39,7.0,11,25.1
385,agrazda01,2019,1,PIT,NL,4,5,15,14,0,...,43,2.0,1.0,5.0,1.364,15,82,15.0,18,73.1
495,alberma01,2019,1,MIL,NL,8,6,67,0,0,...,34,2.0,2.0,8.0,1.374,67,53,8.0,29,59.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56129,yateski01,2019,1,SDN,NL,0,5,60,0,0,...,14,1.0,1.0,4.0,0.890,60,41,2.0,13,60.2
56179,ynoaga01,2019,1,BAL,AL,1,10,36,13,0,...,77,0.0,1.0,15.0,1.373,36,126,29.0,26,110.2
56233,youngal01,2019,1,ARI,NL,7,5,17,15,0,...,40,3.0,0.0,5.0,1.188,17,72,14.0,27,83.1
56497,zeuchtj01,2019,1,TOR,AL,1,2,5,3,0,...,13,0.0,0.0,1.0,1.456,5,22,2.0,11,22.2


## 2 - Predict MVP Vote Pts for 2019

### RANDOM FOREST

In [52]:
# Predict on 2019 data
batt_vote_pts_pred = forest_batting_model.predict(batting_2019[batting_data_columns])
pitch_vote_pts_pred = forest_pitching_model.predict(pitching_2019[pitching_data_columns])

# print("Predicted Batting Vote Pts:\n", batt_vote_pts_pred)
batting_pts = dict(zip(batting_2019['playerID'],batt_vote_pts_pred))
sorted_batting = dict(sorted(batting_pts.items(), key=lambda x: x[1], reverse=True))
print(sorted_batting)
# print("Predicted Pitching Vote Pts:\n", pitch_vote_pts_pred)
pitching_pts = dict(zip(pitching_2019['playerID'], pitch_vote_pts_pred))
sorted_pitching = dict(sorted(pitching_pts.items(), key=lambda x: x[1], reverse=True))
print(sorted_pitching)

{'adamewi01': 291.64, 'ahmedni01': 291.64, 'alfarjo01': 291.64, 'beatyma01': 291.64, 'biggica01': 291.64, 'boteda01': 291.64, 'bradlja02': 291.64, 'dejonpa01': 291.64, 'dixonbr01': 291.64, 'fowlede01': 291.64, 'goodrni01': 291.64, 'heywaja01': 291.64, 'jonesja07': 291.64, 'mcmahry01': 291.64, 'myerswi01': 291.64, 'odorro01': 291.64, 'pillake01': 291.64, 'roblevi01': 291.64, 'santado01': 291.64, 'senzeni01': 291.64, 'swansda01': 291.64, 'vanmejo01': 291.64, 'vottojo01': 291.64, 'calhoko01': 291.57, 'goodwbr01': 291.57, 'hanigmi01': 291.57, 'hernate01': 291.57, 'kingesc01': 291.57, 'machama01': 291.57, 'ozunama01': 291.57, 'puigya01': 291.57, 'renfrhu01': 291.57, 'tayloch03': 291.57, 'doziebr01': 291.22, 'longoev01': 291.22, 'pujolal01': 291.22, 'severpe01': 291.22, 'beltbr01': 291.15, 'carpema01': 291.15, 'desmoia01': 291.15, 'chavimi01': 291.07, 'grichra01': 291.07, 'jonesad01': 291.07, 'piscost01': 291.07, 'seageky01': 291.0, 'walkene01': 290.73, 'chiriro01': 289.18, 'frazito01': 289.

/Users/johnmccarthy/opt/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/johnmccarthy/opt/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


### LIN REG


In [53]:
# Predict on 2019 data
batt_vote_pts_pred = lin_reg_batting_model.predict(batting_2019[batting_data_columns])
pitch_vote_pts_pred = lin_reg_pitching_model.predict(pitching_2019[pitching_data_columns])

# print("Predicted Batting Vote Pts:\n", batt_vote_pts_pred)
batting_pts = dict(zip(batting_2019['playerID'],batt_vote_pts_pred))
sorted_batting = dict(sorted(batting_pts.items(), key=lambda x: x[1], reverse=True))
print(sorted_batting)
# print("Predicted Pitching Vote Pts:\n", pitch_vote_pts_pred)
pitching_pts = dict(zip(pitching_2019['playerID'], pitch_vote_pts_pred))
sorted_pitching = dict(sorted(pitching_pts.items(), key=lambda x: x[1], reverse=True))
print(sorted_pitching)

{'joycema01': -10825.870145255582, 'solando01': -12333.419528189688, 'casteni01': -12766.094989457424, 'adriaeh01': -12902.503525572161, 'vargail01': -12930.555972805048, 'diazal02': -13597.446225548012, 'aquinar01': -13765.6822996938, 'casalcu01': -13775.683150412122, 'luplojo01': -14064.349033988134, 'locasti01': -14109.188105555651, 'heredgu01': -14363.787027435872, 'kineris01': -14453.14554745672, 'mccutan01': -14744.94476097253, 'martiru01': -14877.194916062248, 'rominau01': -14934.846285260915, 'maybica01': -14944.73423731741, 'russead02': -14969.650602460362, 'haselad01': -15038.185382065396, 'frazicl01': -15061.904603554629, 'mejiafr01': -15138.247061924827, 'roberda10': -15272.799154027716, 'ervinph01': -15286.601258842518, 'cordery01': -15370.963485579809, 'aguilje01': -15515.481286977956, 'vanmejo01': -15530.319518599257, 'uriaslu01': -15642.80259717625, 'nolaau01': -15732.329369633162, 'hicksaa01': -15734.072946024453, 'engelad01': -15754.032475183734, 'whitety01': -15773.7

/Users/johnmccarthy/opt/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/Users/johnmccarthy/opt/anaconda3/envs/ds/lib/python3.9/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


### XGBOOST

In [54]:
# Predict on 2019 data
dbatting = xgb.DMatrix(batting_2019[batting_data_columns])
dpitching = xgb.DMatrix(pitching_2019[pitching_data_columns])

batt_vote_pts_pred = xgb_batting_model.predict(dbatting)
pitch_vote_pts_pred = xgb_pitching_model.predict(dpitching)

# print("Predicted Batting Vote Pts:\n", batt_vote_pts_pred)
batting_pts = dict(zip(batting_2019['playerID'],batt_vote_pts_pred))
sorted_batting = dict(sorted(batting_pts.items(), key=lambda x: x[1], reverse=True))
print(sorted_batting)
# print("Predicted Pitching Vote Pts:\n", pitch_vote_pts_pred)
pitching_pts = dict(zip(pitching_2019['playerID'], pitch_vote_pts_pred))
sorted_pitching = dict(sorted(pitching_pts.items(), key=lambda x: x[1], reverse=True))
print(sorted_pitching)

{'yelicch01': 173.3005, 'bellico01': 171.79803, 'troutmi01': 171.79803, 'marteke01': 170.96478, 'anderti01': 170.80199, 'kendrho01': 170.77188, 'buxtoby01': 169.5237, 'desmoia01': 169.5237, 'kingesc01': 169.5237, 'puigya01': 169.5237, 'rosared01': 169.5237, 'sandopa01': 169.5237, 'santaan02': 169.5237, 'schoojo01': 169.5237, 'vogtst01': 169.5237, 'alonspe01': 169.46231, 'arenano01': 169.46231, 'bregmal01': 169.46231, 'gallojo01': 169.46231, 'rendoan01': 169.46231, 'springe01': 169.46231, 'tatisfe02': 169.46231, 'abreujo02': 169.29948, 'acunaro01': 169.29948, 'albieoz01': 169.29948, 'altuvjo01': 169.29948, 'baezja01': 169.29948, 'bettsmo01': 169.29948, 'bogaexa01': 169.29948, 'brantmi02': 169.29948, 'braunry02': 169.29948, 'bryankr01': 169.29948, 'canhama01': 169.29948, 'chapmma01': 169.29948, 'confomi01': 169.29948, 'contrwi01': 169.29948, 'dahlda01': 169.29948, 'davisjd01': 169.29948, 'deverra01': 169.29948, 'diazya01': 169.29948, 'donaljo02': 169.29948, 'doziehu01': 169.29948, 'edman

In [55]:
for i in range(100):
  k, v = list(sorted_batting.items())[i]
  print(k)

yelicch01
bellico01
troutmi01
marteke01
anderti01
kendrho01
buxtoby01
desmoia01
kingesc01
puigya01
rosared01
sandopa01
santaan02
schoojo01
vogtst01
alonspe01
arenano01
bregmal01
gallojo01
rendoan01
springe01
tatisfe02
abreujo02
acunaro01
albieoz01
altuvjo01
baezja01
bettsmo01
bogaexa01
brantmi02
braunry02
bryankr01
canhama01
chapmma01
confomi01
contrwi01
dahlda01
davisjd01
deverra01
diazya01
donaljo02
doziehu01
edmanto01
escobed01
frazicl01
freemfr01
gardnbr01
goldspa01
gourryu01
gurrilo01
harpebr03
judgeaa01
keplema01
laurera01
lemahdj01
lindofr01
lowebr01
luplojo01
mancitr01
martest01
martijd02
maybica01
mcneije01
meadoau01
moncayo01
morelmi01
moustmi01
muncyma01
murphto04
ohtansh01
pederjo01
pencehu01
polanjo01
ramirjo01
realmjt01
reynobr01
rizzoan01
santaca01
santada01
schwaky01
seageco01
semiema01
sogarer01
sotoju01
storytr01
tauchmi01
thameer01
torregl01
turneju01
turnetr01
urshegi01
vazquch01
verdual01
walkech02
yastrmi01
aquinar01
blackch02
correca01
hiurake01
solerjo01


In [56]:
for i in range(20):
  k, v = list(sorted_pitching.items())[i]
  print(k)

adamja01
adamsau02
adamsch01
agrazda01
alberma01
alcansa01
alcanvi01
alexaty01
allarko01
allenlo01
alvarjo02
alvarjo03
anderbr04
anderch01
anderju01
anderni01
andersh01
andrima01
archech01
armstsh01
